In [1]:
from pathlib import Path
import os
import sys

# Setup for importing custom modules.
sys.path.insert(0, str(Path(os.getcwd()).parent.parent))

from src.common.file_io_helper import incriment_file

In [2]:
from typing import Any

import numpy as np

from matplotlib import animation
from matplotlib import pyplot as plt

from scipy.linalg import kron
from scipy.integrate import solve_ivp
from scipy.fftpack import fft2, ifft2

from IPython.display import HTML

%matplotlib notebook

$$
\begin{align*}
\frac {\partial}{\partial t} \left(\begin{align*} u \\ v \end{align*}\right)
    &=
    \left( \begin{align*} &\lambda (A) &-\omega (A) \\ &\omega (A) &\lambda (A) \end{align*} \right)
    \left( \begin{align*} u \\ v \end{align*} \right)
    + D \nabla^2 \left( \begin{align*} u \\ v \end{align*} \right) \\
    &=
    \left( \begin{align*} &1 - A^2 &\beta A^2 \\ &-\beta A^2 & 1 - A^2 \end{align*} \right)
    \left( \begin{align*} u \\ v \end{align*} \right)
    + D \nabla^2 \left( \begin{align*} u \\ v \end{align*} \right)
\end{align*}
$$
with $A^2 = u^2 + v^2$.
Furthermore this results in the following system
$$
U_t = \lambda(A)U - \omega(A)V + D_1 \nabla^2 U \\
V_t = \omega(A)U + \lambda(A)V + D_2 \nabla^2 V
$$
where $\lambda(A) = 1 - A^2$ and $\omega(A) = -\beta A^2$.

In [3]:
def problem_1():
    m = 3
    D1 = 0.1
    D2 = 0.1
    beta = 1
    tspan = np.arange(0, 20.5, .5)

    # Define parameters
    Lx, Ly = 20, 20
    nx, ny = 64, 64
    N = nx * ny

    # Define spatial domain and initial conditions
    x2 = np.linspace(-Lx/2, Lx/2, nx + 1)
    x = x2[:nx]
    y2 = np.linspace(-Ly/2, Ly/2, ny + 1)
    y = y2[:ny]
    X, Y = np.meshgrid(x, y)

    u0 = lambda X, Y: np.tanh(np.sqrt(X**2 + Y**2)) * np.cos(m*np.angle( X + 1j*Y) - np.sqrt(X**2 + Y**2))
    v0 = lambda X, Y: np.tanh(np.sqrt(X**2 + Y**2)) * np.sin(m*np.angle( X + 1j*Y) - np.sqrt(X**2 + Y**2))

    ut0 = fft2(u0(X, Y))
    vt0 = fft2(v0(X, Y))

    uvt0 = np.hstack( (ut0.reshape(N), vt0.reshape(N)) )

    # Define spectral k values
    kx = (2 * np.pi / Lx) * np.concatenate((np.arange(0, nx/2), np.arange(-nx/2, 0)))
    kx[0] = 1e-6
    ky = (2 * np.pi / Ly) * np.concatenate((np.arange(0, ny/2), np.arange(-ny/2, 0)))
    ky[0] = 1e-6
    KX, KY = np.meshgrid(kx, ky)
    K = KX**2 + KY**2

    # Define the ODE system
    def spc_rhs(t, uvt):
        ut_vec, vt_vec = np.split(uvt, [N])

        ut = ut_vec.reshape((nx,ny))
        vt = vt_vec.reshape((nx,ny))

        u = ifft2(ut)
        v = ifft2(vt)

        A = u*u + v*v
        
        lam = 1 - A
        omega = -beta*A

        U = (- D1 * K * ut + fft2(lam*u - omega*v)).reshape(N)
        V = (- D2 * K * vt + fft2(omega*u + lam*v)).reshape(N)

        return np.hstack((U, V))

    sol = solve_ivp(spc_rhs, t_span=(tspan[0], tspan[-1]), y0=uvt0, t_eval=tspan)

    animation.writer = animation.writers['ffmpeg']

    plt.ioff()
    fig = plt.figure()
    ax = fig.add_subplot(111)

    # write the update function, specifically including the ax.clear() function this was important.
    def update(i):
        ax.clear()
        ax.pcolor(np.real(ifft2(sol.y[:N,i].reshape((nx, ny)))), cmap='bwr')
        ax.set_title("Reaction Diffussion with Fourier")
        return ax

    ani = animation.FuncAnimation(fig, update, frames=range(len(sol.t)), interval=125)


    
    file_name = incriment_file("reaction_diffusion_fourier.gif", "visuals")
    ani.save(file_name, writer='pillow')

    # Save as MP4
    file_name_mp4 = incriment_file("reaction_diffusion_fourier.mp4", "visuals")
    writer = animation.writers['ffmpeg']
    writer = writer(metadata=dict(artist='Hunter Lybbert'), fps=25)
    ani.save(file_name_mp4, writer=writer)
    
    return sol, ani

In [4]:
sol_1, ani_1 = problem_1()

In [5]:
A1 = sol_1.y

In [6]:
def problem_2():
    def cheb(N):
        if N==0: 
            D = 0
            x = 1
        else:
            n = np.arange(0, N+1)
            x = np.cos(np.pi*n/N).reshape(N+1,1) 
            c = (np.hstack(( [2.], np.ones(N-1), [2.]))*(-1)**n).reshape(N+1,1)
            X = np.tile(x,(1,N+1))
            dX = X - X.T
            D = np.dot(c,1./c.T)/(dX+np.eye(N+1))
            D -= np.diag(np.sum(D.T,axis=0))
        return D, x.reshape(N+1)

    m = 1
    D1 = 0.1
    D2 = 0.1
    beta = 1
    tspan = np.arange(0, 20.5, .5)

    N = 30
    D, x = cheb(N)
    D[N, :] = 0
    D[0, :] = 0
    D_xx = np.dot(D, D)/((20/2)**2)
    y = x

    N2 = (N + 1)*(N + 1)

    I = np.eye(len(D_xx))
    L = kron(I, D_xx) + kron(D_xx, I)  # 2D Laplacian

    X, Y = np.meshgrid(x*(20/2), y*(20/2))
    # X = X * (20/2)
    # Y = Y * (20/2)

    u0 = lambda X, Y: np.tanh(np.sqrt(X**2 + Y**2)) * np.cos(m*np.angle( X + 1j*Y) - np.sqrt(X**2 + Y**2))
    v0 = lambda X, Y: np.tanh(np.sqrt(X**2 + Y**2)) * np.sin(m*np.angle( X + 1j*Y) - np.sqrt(X**2 + Y**2))

    uv0 = np.hstack( (u0(X, Y).reshape(N2), v0(X, Y).reshape(N2)) )

    # plt.imshow(uv0[:N2].reshape((N + 1, N+1)))
    # plt.show()

    def reaction_diff_rhs(t, uv):
        u, v = np.split(uv, [N2])

        A = u*u + v*v
        
        lam = 1 - A
        omega = -beta*A

        U = D1 * np.dot(L, u) + lam*u - omega*v
        V = D2 * np.dot(L, v) + omega*u + lam*v

        return np.hstack((U, V))
    

    sol = solve_ivp(reaction_diff_rhs, t_span=(tspan[0], tspan[-1]), y0=uv0, t_eval=tspan)

    animation.writer = animation.writers['ffmpeg']

    plt.ioff()
    fig = plt.figure()
    ax = fig.add_subplot(111)

    # write the update function, specifically including the ax.clear() function this was important.
    def update(i):
        ax.clear()
        ax.pcolor(sol.y[:N2,i].reshape((N+1, N+1)), cmap='bwr')
        ax.set_title("Reaction Diffussion with Chebyshev")
        return ax

    ani = animation.FuncAnimation(fig, update, frames=range(len(sol.t)), interval=125)
    
    file_name = incriment_file("reaction_diffusion_cheb.gif", "visuals")
    ani.save(file_name, writer='pillow')

    # Save as MP4
    file_name_mp4 = incriment_file("reaction_diffusion_cheb.mp4", "visuals")
    writer = animation.writers['ffmpeg']
    writer = writer(metadata=dict(artist='Hunter Lybbert'), fps=25)
    ani.save(file_name_mp4, writer=writer)
    
    return sol, ani


In [7]:
sol_2, ani_2 = problem_2()

In [8]:
A2 = sol_2.y